# Universal Powerpoint Translator

- Created by John Tan Chong Min on 14 Jul 2025

- This uses parse_yaml_async to translate a powerpoint from to your desired language
- Works for text in text boxes and text in tables
- Put your POWERPOINT_NAME (without the extension) and LANGUAGE in the variables

In [1]:
!pip install --quiet python-pptx python-dotenv openai agentjo

In [6]:
import os
import re
import asyncio
from dotenv import load_dotenv
from pptx import Presentation
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.enum.dml import MSO_COLOR_TYPE
from openai import AsyncOpenAI
from agentjo import parse_yaml_async

# give the name without the .ppt or .pptx extension
POWERPOINT_NAME = "MemOS"
# target language for translation, e.g. "French", "Spanish", "German"
LANGUAGE = "Japanese"

# Load your OpenAI API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = AsyncOpenAI(api_key=api_key)

async def llm_async(system_prompt: str, user_prompt: str) -> str:
    resp = await client.chat.completions.create(
        model="gpt-4.1",
        temperature=0.0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_prompt},
        ],
    )
    return resp.choices[0].message.content

async def translate_text(text: str, language: str) -> str:
    schema = {"Translated Text": f"Text translated into {language}, type: str"}
    system_prompt = (
        f"You are a translation assistant. Translate the following text into {language}. "
        "Preserve all punctuation and delimiters exactly as in the original. "
        "Produce exactly one field: Translated Text."
    )
    parsed = await parse_yaml_async(
        system_prompt=system_prompt,
        user_prompt=f"Text:\n{text}",
        output_format=schema,
        llm=llm_async,
    )
    return parsed.get("Translated Text", "").strip()

async def process_slide(slide, language):
    notes_slide = slide.notes_slide
    notes_tf = notes_slide.notes_text_frame
    notes_tf.clear()

    # Collect all text frames and table cells, sorted by vertical position
    frames = []
    for shape in slide.shapes:
        if shape.has_text_frame:
            frames.append((shape.top, shape.text_frame))
        elif shape.has_table:
            for row in shape.table.rows:
                for cell in row.cells:
                    frames.append((shape.top, cell.text_frame))
    frames.sort(key=lambda x: x[0])

    for _, tf in frames:
        original_text = tf.text or ""
        if not original_text.strip():
            continue

        # Capture formatting and runs
        paras = []
        for para in tf.paragraphs:
            p_fmt = {
                'alignment': para.alignment,
                'level':     para.level,
                'space_before': para.space_before,
                'space_after':  para.space_after,
            }
            runs = []
            orig_concat = ''
            for run in para.runs:
                r_fmt = {
                    'name':      run.font.name,
                    'size':      run.font.size,
                    'bold':      run.font.bold,
                    'italic':    run.font.italic,
                    'underline': run.font.underline,
                    'color':     run.font.color.rgb if run.font.color.type == MSO_COLOR_TYPE.RGB else None,
                }
                runs.append((run.text, r_fmt))
                orig_concat += run.text
            paras.append((p_fmt, runs, orig_concat))

        # Translate runs and strip any leading/trailing whitespace
        translations = []
        for _, runs, _ in paras:
            for text, _ in runs:
                if re.fullmatch(r"[\W_]+", text):
                    translations.append(text)
                else:
                    translated = await translate_text(text, language)
                    translations.append(translated)

        # Clear frame and rebuild
        tf.clear()
        idx = 0
        for p_fmt, runs, orig in paras:
            new_para = tf.add_paragraph()
            # restore paragraph formatting
            if p_fmt['alignment'] is not None:
                new_para.alignment = p_fmt['alignment']
            if p_fmt['level'] is not None:
                new_para.level = p_fmt['level']
            if p_fmt['space_before'] is not None:
                new_para.space_before = p_fmt['space_before']
            if p_fmt['space_after'] is not None:
                new_para.space_after = p_fmt['space_after']

            text_accum = ''
            for _, r_fmt in runs:
                new_run = new_para.add_run()
                txt = translations[idx]
                # strip leading newlines from each run
                txt = txt.lstrip('\n')
                new_run.text = txt
                text_accum += txt
                idx += 1
                # restore run formatting
                if r_fmt['name']:      new_run.font.name      = r_fmt['name']
                if r_fmt['size']:      new_run.font.size      = r_fmt['size']
                if r_fmt['bold'] is not None:      new_run.font.bold      = r_fmt['bold']
                if r_fmt['italic'] is not None:    new_run.font.italic    = r_fmt['italic']
                if r_fmt['underline'] is not None: new_run.font.underline = r_fmt['underline']
                if r_fmt['color'] is not None:     new_run.font.color.rgb = r_fmt['color']

            # add listener note without leading newlines
            note_para = notes_tf.add_paragraph()
            note_para.text = f"{text_accum.strip()} [{orig.strip()}]"

        # Remove any leading blank paragraph
        if tf.paragraphs and not tf.paragraphs[0].text.strip():
            tf._element.remove(tf.paragraphs[0]._p)

        # autofit
        tf.word_wrap = True
        tf.auto_size  = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE
        try:
            tf.fit_text()
        except Exception:
            pass

async def translate_pptx_async(input_path: str, output_path: str, language: str):
    prs = Presentation(input_path)
    await asyncio.gather(*(process_slide(s, language) for s in prs.slides))
    prs.save(output_path)

async def run_translation():
    infile = f"{POWERPOINT_NAME}.pptx"
    if not os.path.isfile(infile):
        raise FileNotFoundError(f"No '{infile}' found in current directory.")
    suffix  = LANGUAGE.replace(' ', '_')
    outfile = f"{POWERPOINT_NAME}_{suffix}.pptx"
    await translate_pptx_async(infile, outfile, LANGUAGE)
    print(f"Saved translated deck as '{outfile}'")

# To run in Jupyter:
await run_translation()

Saved translated deck as 'MemOS_Japanese.pptx'
